In [1]:
import pandas as pd

In [135]:
movies_meta = pd.read_csv("datasets/the-movies-dataset/movies_metadata.csv")

# filter out foreign movies (not en or de)
movies_meta = movies_meta[movies_meta.original_language.isin(["en", "de"])]

# filter movies with NA in critical columns
for column in ["imdb_id", "title", "release_date", "popularity"]:
    movies_meta = movies_meta[movies_meta[column].isna() == False]

# filter adult movies
movies_meta = movies_meta[movies_meta.adult == "False"]

# sort the list by popularity
movies_meta["popularity"] = pd.to_numeric(movies_meta.popularity)
movies_meta = movies_meta.sort_values("popularity", ascending=False)

# drop unnecessary columns
movies_meta = movies_meta.drop(["adult", "belongs_to_collection", "id"], axis=1)

movies_meta.head(2)

/home/roland/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
30700,74000000,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...",http://www.minionsmovie.com/,tt2293640,en,Minions,"Minions Stuart, Kevin and Bob are recruited by...",547.488298,/q0R4crx2SehcEEQEkYObktdeFy.jpg,"[{'name': 'Universal Pictures', 'id': 33}, {'n...",...,2015-06-17,1.156731e+09,91.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"Before Gru, they had a history of bad bosses",Minions,False,6.4,4729.0
33356,149000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.warnerbros.com/wonder-woman,tt0451279,en,Wonder Woman,An Amazon princess comes to the world of Man t...,294.337037,/imekS7f1OuHyUP2LAiTEM0zBzUz.jpg,"[{'name': 'Dune Entertainment', 'id': 444}, {'...",...,2017-05-30,8.205804e+08,141.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}, {'iso...",Released,Power. Grace. Wisdom. Wonder.,Wonder Woman,False,7.2,5025.0


In [136]:
movies_meta.release_date.isna().sum()

0

In [137]:
movies_meta.popularity.value_counts()

0.000001    41
0.001177    32
0.000000    29
0.000308    28
0.000220    24
            ..
0.441986     1
1.007960     1
1.843645     1
0.007675     1
0.750000     1
Name: popularity, Length: 32307, dtype: int64

In [138]:
movies_meta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33266 entries, 30700 to 24386
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                33266 non-null  object 
 1   genres                33266 non-null  object 
 2   homepage              6326 non-null   object 
 3   imdb_id               33266 non-null  object 
 4   original_language     33266 non-null  object 
 5   original_title        33266 non-null  object 
 6   overview              33082 non-null  object 
 7   popularity            33266 non-null  float64
 8   poster_path           32972 non-null  object 
 9   production_companies  33266 non-null  object 
 10  production_countries  33266 non-null  object 
 11  release_date          33266 non-null  object 
 12  revenue               33266 non-null  float64
 13  runtime               33212 non-null  float64
 14  spoken_languages      33266 non-null  object 
 15  status         

In [150]:
def assign_date_bucket(row):
    as_dt = pd.to_datetime(row["release_date"])
    year = as_dt.year
    
    # before 1990
    if year < 1990:
        return 0
    
    # 1990-2005
    elif year < 2005:
        return 1
    
    # 2005-2020
    return 2

movies_meta["date_segment"] = movies_meta.apply(assign_date_bucket, axis=1)
movies_meta.head(3)

,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,date_segment
30700,74000000,"[{'id': 10751, 'name': 'Family'}, {'id': 16, '...",http://www.minionsmovie.com/,tt2293640,en,Minions,"Minions Stuart, Kevin and Bob are recruited by...",547.488298,/q0R4crx2SehcEEQEkYObktdeFy.jpg,"[{'name': 'Universal Pictures', 'id': 33}, {'n...",...,1.156731e+09,91.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"Before Gru, they had a history of bad bosses",Minions,False,6.4,4729.0,2
33356,149000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.warnerbros.com/wonder-woman,tt0451279,en,Wonder Woman,An Amazon princess comes to the world of Man t...,294.337037,/imekS7f1OuHyUP2LAiTEM0zBzUz.jpg,"[{'name': 'Dune Entertainment', 'id': 444}, {'...",...,8.205804e+08,141.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}, {'iso...",Released,Power. Grace. Wisdom. Wonder.,Wonder Woman,False,7.2,5025.0,2
42222,160000000,"[{'id': 10751, 'name': 'Family'}, {'id': 14, '...",http://movies.disney.com/beauty-and-the-beast-...,tt2771200,en,Beauty and the Beast,A live-action adaptation of Disney's version o...,287.253654,/tWqifoYuwLETmmasnGHO7xBjEtt.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}, {'...",...,1.262886e+09,129.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Be our guest.,Beauty and the Beast,False,6.8,5530.0,2


In [146]:
movies_meta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33266 entries, 30700 to 24386
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   budget                33266 non-null  object        
 1   genres                33266 non-null  object        
 2   homepage              6326 non-null   object        
 3   imdb_id               33266 non-null  object        
 4   original_language     33266 non-null  object        
 5   original_title        33266 non-null  object        
 6   overview              33082 non-null  object        
 7   popularity            33266 non-null  float64       
 8   poster_path           32972 non-null  object        
 9   production_companies  33266 non-null  object        
 10  production_countries  33266 non-null  object        
 11  release_date          33266 non-null  object        
 12  revenue               33266 non-null  float64       
 13  runtime     